# MNIST on the Jetson Nano (JN)
The model below is an example of a very simple network for handwritten digits recognition using ReLU6 as an activation layer. The ReLU6 layer is not directly supported in the TensorRT APIs. Therefore you will provide a CUDA implementation of the corresponding plugin.

Before this step, you have to train a network and import the .pb file in JN. You have two options:

1.   train and then use the model below in the JN;
2.   train a network of your choice also using ReLU6 (after flattening) and - if you feel like - possibly other TensorRT unsopported layers; then use your network in the JN.


First of all, copy this Notebook in your Google Drive: File -> Save a copy to Drive. If you choose the first option above you can then directly run the Notebook and download the .pb file. If you choose the second option, you have to first replace the network with yours and validate it (i.e. it must be at least 98% accurate on the MNIST dataset).

Note that to speed up this part we have chosen a very simple network (an MLP) and we are doing a quick-an-dirty suboptimal training. This is in fact not the goal here. Instead we want to quickly deploy the network on the JN and run inference with it.


In [ ]:

import tensorflow as tf
import numpy as np
import os

# Needed to run the example with tensorflow2
with tf.Graph().as_default():
  #WORKING_DIR = os.environ.get("TRT_WORKING_DIR") or os.path.dirname(os.path.realpath(__file__))
  WORKING_DIR = "./"
  MODEL_DIR = os.path.join(
      WORKING_DIR,
      'models'
  )


  def load_data():
      mnist = tf.compat.v1.keras.datasets.mnist
      (x_train, y_train),(x_test, y_test) = mnist.load_data()
      x_train, x_test = x_train / 255.0, x_test / 255.0
      x_train = np.reshape(x_train, (-1, 1, 28, 28))
      x_test = np.reshape(x_test, (-1, 1, 28, 28))
      return x_train, y_train, x_test, y_test

  def build_model():
      # Create the keras model
      model = tf.keras.models.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape=[1, 28, 28], name="InputLayer"))
      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(512))
      model.add(tf.keras.layers.Activation(activation=tf.nn.relu6, name="ReLU6"))
      model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax, name="OutputLayer"))
      return model

  def train_model():
      # Build and compile model
      model = build_model()
      model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

      # Load data
      x_train, y_train, x_test, y_test = load_data()

      # Train the model on the data
      model.fit(
          x_train, y_train,
          epochs = 10,
          verbose = 1
      )

      # Evaluate the model on test data
      test_loss, test_acc = model.evaluate(x_test, y_test)
      print("Test loss: {}\nTest accuracy: {}".format(test_loss, test_acc))

      return model

  def maybe_mkdir(dir_path):
      if not os.path.exists(dir_path):
          os.makedirs(dir_path)

  def save_model(model):
      output_names = model.output.op.name
      sess = tf.keras.backend.get_session()

      graphdef = sess.graph.as_graph_def()

      frozen_graph = tf.graph_util.convert_variables_to_constants(sess, graphdef, [output_names])
      frozen_graph = tf.graph_util.remove_training_nodes(frozen_graph)

      # Make directory to save model in if it doesn't exist already
      maybe_mkdir(MODEL_DIR)

      model_path = os.path.join(MODEL_DIR, "trained_lenet5.pb")
      with open(model_path, "wb") as ofile:
          ofile.write(frozen_graph.SerializeToString())


  model = train_model()
  save_model(model)




In [ ]:
import tensorflow as tf
import numpy as np
import os

with tf.Graph().as_default():

  def build_model():
      model = tf.keras.models.Sequential()
      model.add(tf.keras.layers.InputLayer(input_shape=[1, 28, 28], name="InputLayer"))
      model.add(tf.keras.layers.Flatten())
      model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu6))
      model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu6))
      model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
      return model

  model = build_model()
  model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

  model.summary()